In [ ]:
import branch_length
import distributions
import optimizers

import altair as alt
import numpy as np
import pandas as pd
import scipy.stats as stats

alt.renderers.enable("notebook")

import importlib 
importlib.reload(branch_length)
importlib.reload(distributions)
importlib.reload(optimizers)

In [ ]:
x_vals = np.linspace(0.1, 5, 40)
x_vals_transpose = np.transpose(np.array([x_vals]))

def plot_functions(f_true, f_approx):
    x_transpose = np.transpose(np.array([x_vals]))
    data = pd.DataFrame({"x": x_vals, "truth": f_true(x_vals_transpose), 
                         "approx": f_approx(x_vals_transpose)})
    return alt.Chart(data.melt(id_vars=["x"])).mark_line().encode(
        x='x',
        y='value',
        color='variable'
    )

def kl_div(f_true, f_approx):
    return {
        "standard": stats.entropy(f_true(x_vals_transpose), f_approx(x_vals_transpose)),
        "reversed": stats.entropy(f_approx(x_vals_transpose), f_true(x_vals_transpose))}

In [ ]:
branch_length_param_count = 1
infer_opt = optimizers.SGD_Server(
    {'loc': branch_length_param_count, 'shape': branch_length_param_count})
stepsz = 0.05
stepsz_dict = {'loc': stepsz, 'shape': stepsz}

# Distribution we wish to approximate-- we pretend this is the "phylogenetic" distribution.
d = distributions.Normal(1)
true_loc = np.array([4.])
true_shape = np.array([1.])
phylo_log_like = lambda x: d.log_prob(x, true_loc, true_shape)
phylo_log_like_grad = lambda x: d.log_prob_grad(x, true_loc, true_shape)

# Variational distribution
q = distributions.Normal(1)
loc = np.array([1.])
shape = np.array([0.5])
q_log_like = lambda x: q.log_prob(x, loc, shape)

In [ ]:
def gradient_step(sample_count):
    global loc, shape
    x = q.sample(loc, shape, sample_count)
    weights = branch_length.like_weights(q, phylo_log_like(x), x, loc, shape)
    loc_grad, shape_grad = branch_length.param_grad(q, weights, phylo_log_like_grad(x), x, loc, shape)
    update_dict = infer_opt.adam(stepsz_dict, {'loc': loc, 'shape': shape}, 
                                 {'loc': loc_grad, 'shape': shape_grad})
    loc += update_dict['loc']
    shape += update_dict['shape']
    return kl_div(phylo_log_like, q_log_like)

In [ ]:
loc = np.array([0.5])
shape = np.array([1.])
results = [kl_div(phylo_log_like, q_log_like)]

for _ in range(200):
    results.append(gradient_step(10))

plot_data = pd.DataFrame(results).reset_index()
    
alt.Chart(
    pd.melt(plot_data, id_vars=["index"], var_name="variant", value_name="KL divergence")
    ).mark_line().encode(
        alt.X("index"),
        alt.Y("KL divergence",
              scale=alt.Scale()),
        color="variant"
    )

In [ ]:
gradient_step(10)
plot_functions(phylo_log_like, q_log_like)

In [ ]:
plot_functions(phylo_log_like, q_log_like)

In [ ]:
x = q.sample(loc, shape, 10)

In [ ]:
weights = branch_length.like_weights(q, phylo_log_like(x), x, loc, shape)
print("weighted:\t", branch_length.param_grad(q, weights, phylo_log_like_grad(x), x, loc, shape))
uniform_weights = np.ones(weights.shape) / len(weights)
print("unweighted:\t", branch_length.param_grad(q, uniform_weights, phylo_log_like_grad(x), x, loc, shape))

In [ ]:
weights

In [ ]:
loc_grad, shape_grad = param_grad_gory(q, weights, phylo_log_like_grad(x), x, loc, shape)

In [ ]:
weights

In [ ]:
x_simple = x.transpose()[0]
df = pd.DataFrame(
    {"x": x_simple, "epsilon": (x_simple - loc)/shape, "weights": weights, 
     "loc_grad": loc_grad.transpose()[0], "shape_grad": shape_grad.transpose()[0]})
df.sort_values("x")

In [ ]:
np.concatenate((x, shape_grad), axis=1)

In [ ]:

def param_grad_gory(q_distribution, weights, phylo_gradient, x, loc, shape):
    """The gradient with respect to the parameters of the variational
    distribution using the reparametrization trick.

    See (7) of the 2018 ICLR paper.
    """
    # Recall that the branch lengths are parameterized in terms of the variational
    # parameters (epsilon is considered fixed) so when we take their derivative we need
    # to take the branch length derivative:
    d_log_prob_ratio = phylo_gradient - q_distribution.log_prob_grad(x, loc, shape)
    # ... and then the reparametrization derivative due to the chain rule.
    d_reparam_loc, d_reparam_shape = q_distribution.reparam_grad(x, loc, shape)
    # This is the derivative with respect to the variational parametrization itself.
    d_q_loc, d_q_shape = q_distribution.log_prob_param_grad(x, loc, shape)
    # Here are the full derivatives, where the first term is the derivative WRT
    # parameters in the reparameterization trick and the second is WRT the actual
    # variational parameterization.
    d_loc = d_log_prob_ratio * d_reparam_loc - d_q_loc
    d_shape = d_log_prob_ratio * d_reparam_shape - d_q_shape
    # Our multiple samples are laid out on axis 0, so this multiplication on the left
    # reweights them.
    #weights = np.ones(weights.shape)/len(weights)
    return d_loc, d_shape


In [ ]:
loc

In [ ]:
shape

In [ ]:
x = q.sample(loc, shape, 100)
weights = branch_length.like_weights(q, phylo_log_like(x), x, loc, shape)
branch_length.param_grad(q, weights, phylo_log_like_grad(x), x, loc, shape)

In [ ]:
df = pd.DataFrame({"x": x[:,0], "weights": weights})
df.sort_values("x", inplace=True)

In [ ]:
alt.Chart(df).mark_line().encode(
        alt.X("x"),
        alt.Y("weights")
    )